In [1]:
import praw
import pandas as pd
import datetime as dt
from psaw import PushshiftAPI
import re

In [2]:
reddit = praw.Reddit(client_id='73IApaJarJRRVA',client_secret='QIET4TRTvxlLWDNFfobmDD7oAfM',user_agent='PD sentiment',username='hll4ce',password='Snsh11nE')
api = PushshiftAPI(reddit)

In [9]:
start_epoch = int(dt.datetime(2012, 1, 1).timestamp()) 

submissions_all_time = list(api.search_submissions(after = start_epoch,
                                                   subreddit = 'Parkinsons',
                                                   filter=['url','author', 'title', 'subreddit'],
                                                   limit = None))
len(submissions_all_time)

submissions_dict = { "title":[], "author":[], "score":[], "id":[], "url":[],"comms_num": [], "created": [], "body":[]}
    
for submission in submissions_all_time:
    
    submissions_dict["title"].append(submission.title)
    submissions_dict["author"].append(str(submission.author))
    submissions_dict["score"].append(submission.score)
    submissions_dict["id"].append(submission.id)
    submissions_dict["url"].append(submission.url)
    submissions_dict["comms_num"].append(submission.num_comments)
    submissions_dict["created"].append(submission.created)
    submissions_dict["body"].append(submission.selftext)
    

submissions_data = pd.DataFrame(submissions_dict)

In [5]:
comments_all_time = list(api.search_comments(after = start_epoch,
                                             subreddit = 'Parkinsons',
                                             limit = None))

comments_dict = {"parent_id":[], 
                 "subreddit_id":[],
                 "author":[],
                 "id":[], 
                 "score":[], 
                 "created": [], 
                 "body":[], 
                 "controversiality":[]}
    
for comment in comments_all_time:
    comments_dict["parent_id"].append(comment.parent_id)
    comments_dict["subreddit_id"].append(comment.subreddit_id)
    comments_dict["author"].append(str(comment.author))
    comments_dict["id"].append(comment.id)
    comments_dict["score"].append(comment.score)
    comments_dict["controversiality"].append(comment.controversiality)
    comments_dict["created"].append(comment.created)
    comments_dict["body"].append(comment.body)

comments_data = pd.DataFrame(comments_dict)

In [74]:
submissions_data.head()

,title,author,score,id,url,comms_num,created,body
0,Erectile dysfunction early sign,Russpow,1,ecsetq,https://www.reddit.com/r/Parkinsons/comments/e...,1,1.576787e+09,"Hello, my father currently has Parkinson’s dis..."
1,Loss of concentration,BookPherq,5,ecnwhm,https://www.reddit.com/r/Parkinsons/comments/e...,1,1.576757e+09,I have been living with YOPD for a few years. ...
2,How to detect Parkinson's with certainty?,Dark_Knight003,4,ecif6j,https://www.reddit.com/r/Parkinsons/comments/e...,8,1.576731e+09,"I am 33, Male and have been experiencing rando..."
3,I have a few questions for people with Parkins...,AgentFN2187,7,ec9h83,https://www.reddit.com/r/Parkinsons/comments/e...,6,1.576683e+09,"1#: Do you ever get ""used"" to the shaking? Th..."
4,Looking for advice about levodopa.,None,2,ec5tmc,https://www.reddit.com/r/Parkinsons/comments/e...,2,1.576663e+09,[deleted]


In [6]:
comments_data.head()

,parent_id,subreddit_id,author,id,score,created,body,controversiality
0,t1_fbejwzg,t5_2u0be,Dark_Knight003,fbg5z2s,1,1.576865e+09,"What's an ""essential tremor""?",0
1,t3_ecif6j,t5_2u0be,Dark_Knight003,fbg5xwb,1,1.576865e+09,Well I have definitely had oscillating movemen...,0
2,t3_e9w9oy,t5_2u0be,hopefulrealist23,fbfu0nl,1,1.576850e+09,How often are you exercising? Do you do any ph...,0
3,t1_fbdfh2h,t5_2u0be,Russpow,fbfb2lp,1,1.576835e+09,Thanks for your reply. That’s good to hear tha...,0
4,t1_fanojls,t5_2u0be,RebYaakov,fbf80zm,1,1.576833e+09,Thanks for the feedback! I do see a great MDS...,0


In [7]:
comments_data["parent_id_tidy"] = [re.sub(".*_", "", x) for x in comments_data["parent_id"]]

In [10]:
# there are some Falses so some ids don't match up from comments to submissions
[x in list(submissions_data["id"]) for x in comments_data["parent_id_tidy"]][1:10]

[True, True, False, False, True, True, False, True, False]

In [11]:
print(len(submissions_data))
print(len(comments_data))

2588
11950


In [12]:
#fix date column
def get_date(created):
    return dt.datetime.fromtimestamp(created)


In [13]:
_timestamp = submissions_data["created"].apply(get_date)

submissions_data = submissions_data.assign(timestamp = _timestamp)

submissions_data.head()

,title,author,score,id,url,comms_num,created,body,timestamp
0,Question about ease-of-use mobile phone holder...,yulespruce,2,ed65y8,https://www.reddit.com/r/Parkinsons/comments/e...,0,1.576851e+09,"Hi /r/Parkinsons,\n\nSo my dad with PD has rec...",2019-12-20 14:15:42
1,Erectile dysfunction early sign,Russpow,1,ecsetq,https://www.reddit.com/r/Parkinsons/comments/e...,2,1.576787e+09,"Hello, my father currently has Parkinson’s dis...",2019-12-19 20:16:10
2,Loss of concentration,BookPherq,10,ecnwhm,https://www.reddit.com/r/Parkinsons/comments/e...,7,1.576757e+09,I have been living with YOPD for a few years. ...,2019-12-19 12:09:40
3,How to detect Parkinson's with certainty?,Dark_Knight003,3,ecif6j,https://www.reddit.com/r/Parkinsons/comments/e...,16,1.576731e+09,"I am 33, Male and have been experiencing rando...",2019-12-19 04:50:30
4,I have a few questions for people with Parkins...,AgentFN2187,7,ec9h83,https://www.reddit.com/r/Parkinsons/comments/e...,6,1.576683e+09,"1#: Do you ever get ""used"" to the shaking? Th...",2019-12-18 15:35:41


In [14]:
_timestamp = comments_data["created"].apply(get_date)

comments_data = comments_data.assign(timestamp = _timestamp)

comments_data.head()

,parent_id,subreddit_id,author,id,score,created,body,controversiality,parent_id_tidy,timestamp
0,t1_fbejwzg,t5_2u0be,Dark_Knight003,fbg5z2s,1,1.576865e+09,"What's an ""essential tremor""?",0,fbejwzg,2019-12-20 18:02:44
1,t3_ecif6j,t5_2u0be,Dark_Knight003,fbg5xwb,1,1.576865e+09,Well I have definitely had oscillating movemen...,0,ecif6j,2019-12-20 18:01:54
2,t3_e9w9oy,t5_2u0be,hopefulrealist23,fbfu0nl,1,1.576850e+09,How often are you exercising? Do you do any ph...,0,e9w9oy,2019-12-20 13:49:37
3,t1_fbdfh2h,t5_2u0be,Russpow,fbfb2lp,1,1.576835e+09,Thanks for your reply. That’s good to hear tha...,0,fbdfh2h,2019-12-20 09:43:00
4,t1_fanojls,t5_2u0be,RebYaakov,fbf80zm,1,1.576833e+09,Thanks for the feedback! I do see a great MDS...,0,fanojls,2019-12-20 09:06:24


In [15]:
submissions_data.to_csv("/Users/leonardhl/Documents/Sentiment-in-PD-Progression/parkinsons_submissions.csv")
comments_data.to_csv("/Users/leonardhl/Documents/Sentiment-in-PD-Progression/parkinsons_comments.csv")